In [2]:
import pandas as pd
import numpy as np

df = pd.read_csv("customer_churn_messy.csv")

df.info()
df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 60 entries, 0 to 59
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   CustomerID  60 non-null     int64  
 1   Age         51 non-null     float64
 2   Gender      60 non-null     object 
 3   Salary      60 non-null     int64  
 4   Churn       60 non-null     object 
dtypes: float64(1), int64(2), object(2)
memory usage: 2.5+ KB


,CustomerID,Age,Gender,Salary,Churn
0,1,25.0,M,50000,No
1,2,34.0,Female,62000,Yes
2,3,NaN,male,58000,No
3,4,45.0,F,1200000,Yes
4,5,29.0,female,54000,No


In [3]:
df = df.drop_duplicates()

In [4]:
df['Gender'] = (
    df['Gender']
    .str.strip()
    .str.lower()
    .replace({
        'm': 'male',
        'f': 'female'
    })
)


In [5]:
df['Gender'].value_counts()

Gender
male      30
female    30
Name: count, dtype: int64

In [6]:
df['Age'] = df['Age'].fillna(df['Age'].median())


In [7]:
Q1 = df['Salary'].quantile(0.25)
Q3 = df['Salary'].quantile(0.75)
IQR = Q3 - Q1

lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR


In [9]:
df['Salary'] = np.clip(df['Salary'], lower_bound, upper_bound)


In [10]:
df = df[(df['Salary'] >= lower_bound) & (df['Salary'] <= upper_bound)]


In [11]:
df = pd.get_dummies(df, columns=['Gender'], drop_first=True)


In [12]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
numeric_cols = ['Age', 'Salary']

df[numeric_cols] = scaler.fit_transform(df[numeric_cols])


In [13]:
df.isnull().sum()
df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 60 entries, 0 to 59
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   CustomerID   60 non-null     int64  
 1   Age          60 non-null     float64
 2   Salary       60 non-null     float64
 3   Churn        60 non-null     object 
 4   Gender_male  60 non-null     bool   
dtypes: bool(1), float64(2), int64(1), object(1)
memory usage: 2.1+ KB


In [15]:
df.to_csv("customer_churn_cleaned_1.csv", index=False)
